In [1]:
# prompt: connect to drive to my folder in which this file is available

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/NLP CLASS/A 1") # Replace with the actual path
print(os.getcwd())

/content/drive/MyDrive/NLP CLASS/A 1


In [4]:
!pip install wikipedia-api
!pip install pandas
import random
import re
import pandas as pd
import wikipediaapi

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15384 sha256=551ad2bb8b2f6cd1d6b37b3a7d77bcc2f5623c362324f3812cf924f6fd910dcf
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built wikipedia-api


In [6]:
wiki_wiki = wikipediaapi.Wikipedia(
    language="en",
    user_agent="MyWikipediaScraper/1.0 (mailto:ankitchahal27@gmail.com)"
)

In [8]:
topics = ["Artificial intelligence", "Climate change", "History of mathematics",
    "Quantum mechanics", "Shakespeare", "Modern architecture",
    "Neural networks", "Renaissance art", "Human evolution",
    "Space exploration", "Blockchain", "Psychology", "Machine learning",
    "Cybersecurity", "Ancient civilizations", "Philosophy of science",
    "Cryptography", "Music theory", "World War II", "Genetics",
    "Medical advancements", "Astronomy", "Linguistics", "Sociology",
    "Economics", "Political philosophy", "Feminism", "Robotics",
    "Environmental ethics", "Game theory"]

In [9]:
# Function to clean and extract text
def clean_wiki_text(text):
    text = re.sub(r'\n+', ' ', text)  # Remove newlines
    text = re.sub(r'\[[^\]]*\]', '', text)  # Remove references like [1], [2]
    text = re.sub(r'\([^)]*\)', '', text)  # Remove text in parentheses
    text = re.sub(r'==.*?==', '', text)  # Remove section headers
    text = text.strip()
    return text

In [10]:
# Function to fetch a Wikipedia article
def get_wikipedia_text(topic):
    page = wiki_wiki.page(topic)
    if not page.exists():
        return None

    text = clean_wiki_text(page.text)

    # Split text into chunks of 100-200 words
    sentences = text.split('. ')
    chunk = []
    dataset = []

    for sentence in sentences:
        chunk.append(sentence)
        word_count = len(' '.join(chunk).split())

        if 100 <= word_count <= 200:  # Ensure each chunk has 100-200 words
            dataset.append(' '.join(chunk))
            chunk = []  # Reset for next chunk

    return dataset

In [11]:
# Collect Wikipedia text samples
wikipedia_texts = []
for topic in topics:
    if len(wikipedia_texts) >= 2000:
        break  # Stop when we reach 2000 samples
    try:
        text_chunks = get_wikipedia_text(topic)
        if text_chunks:
            wikipedia_texts.extend(text_chunks)
    except Exception as e:
        print(f"Error fetching topic {topic}: {e}")

In [12]:
# Create a DataFrame
df_wikipedia = pd.DataFrame(wikipedia_texts, columns=["text"])
df_wikipedia["label"] = 0  # Label 0 for human-written text


In [13]:
# Save as CSV
df_wikipedia.to_csv("wikipedia_texts.csv", index=False)

In [15]:
print(f"Successfully collected {len(df_wikipedia)} Wikipedia samples!")

Successfully collected 1909 Wikipedia samples!


In [17]:
import requests
from bs4 import BeautifulSoup
import time

In [53]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

# Use the 61 collected URLs
article_urls = [
    "https://www.aljazeera.com/economy/2025/3/14/eu-parliament-rocked-by-corruption-investigation-linked-to-chinas-huawei",
    "https://www.aljazeera.com/opinions/2025/3/7/the-world-cannot-ignore-trumps-death-threat-to-the-people-of-gaza",
    "https://www.aljazeera.com/opinions/2025/3/8/a-tribute-to-um-adnan",
    "https://www.aljazeera.com/economy/2025/3/14/consumer-sentiment-plunges-as-us-frightened-of-sharply-higher-prices",
    "https://www.aljazeera.com/opinions/2025/3/9/america-needs-donald-trump-right-now-heres-why",
    "https://www.aljazeera.com/economy/2025/3/13/us-public-souring-on-trumps-handling-of-economy-polls-show",
    "https://www.aljazeera.com/news/2025/3/15/israel-kills-at-least-nine-palestinians-including-journalists-in-gaza",
    "https://www.aljazeera.com/economy/2025/3/11/even-as-us-slashes-jobs-it-is-the-calm-before-the-storm-economists-warn",
    "https://www.aljazeera.com/news/2025/3/15/syrians-mark-revolution-anniversary-with-roses-songs-and-tight-security",
    "https://www.aljazeera.com/features/2025/3/9/searching-for-africas-lost-world-war-soldiers-one-grave-at-a-time",
    "https://www.aljazeera.com/opinions/2025/3/12/arrests-of-columbia-pro-palestine-activists-will-not-save-israels-image",
    "https://www.aljazeera.com/features/2025/3/9/beauty-in-gaza-noors-tent-salon-in-the-rubble",
    "https://www.aljazeera.com/opinions/2025/3/6/a-new-muslim-and-african-ban-by-trump-can-and-must-be-prevented",
    "https://www.aljazeera.com/features/2025/3/14/is-trump-trying-to-pull-putin-away-from-china-and-can-it-work",
    "https://www.aljazeera.com/sports/2025/3/14/indian-premier-league-2025-axar-patel-named-delhi-capitals-captain",
    "https://www.aljazeera.com/news/2025/3/11/trump-says-imposing-50-percent-tariffs-on-canadian-steel-aluminium",
    "https://www.aljazeera.com/economy/2025/3/10/be-quiet-small-man-musk-clashes-with-polish-pm-over-starlink-in-ukraine",
    "https://www.aljazeera.com/sports/2025/3/13/australian-f1-gp-lewis-hamilton-itching-to-make-ferrari-debut",
    "https://www.aljazeera.com/sports/2025/3/16/pakistan-lose-first-t20-against-new-zealand-despite-khushdil-shah-heroics",
    "https://www.aljazeera.com/news/2025/3/15/ailing-pope-francis-works-on-signature-reform-from-hospital-as-he-recovers",
    "https://www.aljazeera.com/features/2025/3/12/a-rain-of-rockets-and-bullets-survivors-of-pakistans-train-hijacking",
    "https://www.aljazeera.com/economy/2025/3/14/meta-to-test-community-notes-using-algorithm-from-elon-musks-x",
    "https://www.aljazeera.com/opinions/2025/3/6/multilateralism-can-and-must-deliver",
    "https://www.aljazeera.com/news/2025/3/15/uks-starmer-says-coalition-to-beef-up-ukraine-security-in-any-peace-deal",
    "https://www.aljazeera.com/features/2025/3/13/palestinians-displaced-by-israeli-aggression-hit-new-levels-of-uncertainty",
    "https://www.aljazeera.com/economy/2025/3/12/bank-of-canada-cuts-interest-rates-warned-country-faces-new-crisis",
    "https://www.aljazeera.com/news/2025/3/13/trumps-steel-aluminium-tariffs-how-are-targeted-countries-responding",
    "https://www.aljazeera.com/news/2025/3/13/people-need-a-break-drc-conflict-reignites-dark-memories-of-congo-wars",
    "https://www.aljazeera.com/news/2025/3/15/huge-power-outage-in-cuba-leaves-millions-in-darkness",
    "https://www.aljazeera.com/opinions/2025/3/15/gazas-silent-epidemic",
    "https://www.aljazeera.com/opinions/2025/3/10/black-palestinian-solidarity-we-need-to-talk-about-what-happened",
    "https://www.aljazeera.com/economy/2025/3/13/elon-musk-wants-to-use-ai-to-run-us-govt-but-experts-say-very-bad-idea",
    "https://www.aljazeera.com/opinions/2025/3/13/trumps-tariffs-may-end-up-blowing-up-the-us-dollar-hegemony",
    "https://www.aljazeera.com/features/2025/3/10/we-just-want-peace-a-pacifist-community-amid-ethiopias-amhara-conflict",
    "https://www.aljazeera.com/economy/2025/3/11/bloodbath-us-stock-market-sheds-1-75tn-after-trumps-recession-remarks",
    "https://www.aljazeera.com/opinions/2025/3/13/trumps-usaid-freeze-must-serve-as-a-wake-up-call-for-africa",
    "https://www.aljazeera.com/news/2025/3/15/inch-by-inch-myanmar-rebels-close-in-on-key-military-base-in-chin-state",
    "https://www.aljazeera.com/news/2025/3/15/thousands-join-antigovernment-protest-in-serbias-capital-belgrade",
    "https://www.aljazeera.com/sports/2025/3/15/atletico-madrid-vs-barcelona-laliga-preview-teams-how-to-follow-stream",
    "https://www.aljazeera.com/news/2025/3/15/deadly-storms-kill-at-least-17-across-three-us-states",
    "https://www.aljazeera.com/sports/2025/3/14/alcarazs-indian-wells-three-peat-bid-on-track-with-draper-next-up",
    "https://www.aljazeera.com/economy/2025/3/14/us-stock-market-takes-another-tumble-as-trump-threatens-tariffs-on-wine",
    "https://www.aljazeera.com/news/2025/3/13/families-of-dutertes-drug-war-victims-grieve-seek-justice-in-philippines",
    "https://www.aljazeera.com/sports/2025/3/14/formula-one-south-africa-bids-to-return-f1-to-continent-where-when-how",
    "https://www.aljazeera.com/news/2025/3/11/why-china-isnt-as-worried-about-trumps-trade-war-as-in-2018",
    "https://www.aljazeera.com/news/2025/3/15/us-launches-major-air-strikes-on-houthi-targets-in-yemen-at-least-9-killed",
    "https://www.aljazeera.com/opinions/2025/3/7/the-real-reason-behind-trumps-tariffs-on-mexico-and-military",
    "https://www.aljazeera.com/news/2025/3/12/closest-target-why-is-donald-trump-so-focused-on-canada",
    "https://www.aljazeera.com/sports/2025/3/15/wpl-2025-final-mumbai-indians-claim-second-title-as-delhi-capitals-despair",
    "https://www.aljazeera.com/sports/2025/3/13/thunder-beat-celtics-to-clinch-nba-playoff-spot",
    "https://www.aljazeera.com/opinions/2025/3/12/trump-wants-to-deport-foreign-students-like-me-universities-must-defy-him",
    "https://www.aljazeera.com/sports/2025/3/14/fernandes-hits-back-at-ratcliffe-over-overpaid-jibe-at-manchester-united"
]


# Headers for requests
HEADERS = {"User-Agent": "Mozilla/5.0"}

def extract_article_text(url):
    """Extracts and cleans article text from an Al Jazeera news page."""
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code != 200:
            print(f"⚠️ Failed to fetch {url} (Status: {response.status_code})")
            return None

        soup = BeautifulSoup(response.text, "html.parser")

        # Locate the main article content
        article_body = soup.find("div", class_="wysiwyg wysiwyg--all-content")
        if not article_body:
            print(f"❌ Article body not found for {url}")
            return None

        paragraphs = article_body.find_all("p")
        text = " ".join([p.get_text() for p in paragraphs])

        # Clean text
        text = re.sub(r'\s+', ' ', text).strip()

        return text

    except Exception as e:
        print(f"Error extracting {url}: {e}")
        return None

def split_into_samples(text, min_words=100, max_words=200):
    """Splits a long text into multiple samples of 100-200 words."""
    words = text.split()
    samples = []

    i = 0
    while i < len(words):
        length = min(max_words, len(words) - i)
        if length < min_words:
            break  # Stop if remaining text is too short
        sample = " ".join(words[i:i + length])
        samples.append(sample)
        i += length  # Move to next section

    return samples

# List to store samples
samples = []

# Process each URL
for url in article_urls:
    print(f"🔍 Scraping: {url}")
    article_text = extract_article_text(url)

    if article_text:
        text_samples = split_into_samples(article_text)
        samples.extend(text_samples)

    time.sleep(1)  # Be respectful with delays

# Create DataFrame
df = pd.DataFrame(samples, columns=["text"])
df["label"] = 0  # Human-written label

# Save to CSV
df.to_csv("aljazeera_news_articles.csv", index=False)

print(f"✅ Successfully collected {len(df)} samples!")

🔍 Scraping: https://www.aljazeera.com/economy/2025/3/14/eu-parliament-rocked-by-corruption-investigation-linked-to-chinas-huawei
🔍 Scraping: https://www.aljazeera.com/opinions/2025/3/7/the-world-cannot-ignore-trumps-death-threat-to-the-people-of-gaza
🔍 Scraping: https://www.aljazeera.com/opinions/2025/3/8/a-tribute-to-um-adnan
🔍 Scraping: https://www.aljazeera.com/economy/2025/3/14/consumer-sentiment-plunges-as-us-frightened-of-sharply-higher-prices
🔍 Scraping: https://www.aljazeera.com/opinions/2025/3/9/america-needs-donald-trump-right-now-heres-why
🔍 Scraping: https://www.aljazeera.com/economy/2025/3/13/us-public-souring-on-trumps-handling-of-economy-polls-show
🔍 Scraping: https://www.aljazeera.com/news/2025/3/15/israel-kills-at-least-nine-palestinians-including-journalists-in-gaza
🔍 Scraping: https://www.aljazeera.com/economy/2025/3/11/even-as-us-slashes-jobs-it-is-the-calm-before-the-storm-economists-warn
🔍 Scraping: https://www.aljazeera.com/news/2025/3/15/syrians-mark-revolution-

### Books

In [66]:
import requests
from bs4 import BeautifulSoup
import csv
import random
import time

# List of popular books by ID (from Gutenberg)
book_ids = [
    1342, 84, 1080, 2600, 6130, 1661, 11, 5200, 2701, 98, 174, 43,
    25344, 16, 1400, 74, 1952, 2852, 829, 1250, 1727, 345, 730, 30254, 1260
]

# Function to extract book text from Gutenberg
def extract_gutenberg_text(book_id):
    """Download and extract book text from Project Gutenberg"""
    try:
        url = f"https://www.gutenberg.org/files/{book_id}/{book_id}-0.txt"
        response = requests.get(url)
        response.raise_for_status()

        text = response.text
        start_index = text.find("*** START OF THIS PROJECT GUTENBERG EBOOK")
        end_index = text.find("*** END OF THIS PROJECT GUTENBERG EBOOK")

        if start_index != -1 and end_index != -1:
            text = text[start_index:end_index]
        elif start_index != -1:
            text = text[start_index:]
        elif end_index != -1:
            text = text[:end_index]

        return text if len(text.split()) > 100 else None  # Ensure valid text

    except Exception as e:
        print(f"❌ Error extracting book {book_id}: {e}")
        return None

# Function to split text into 100-200 word samples
def split_into_samples(text, min_words=100, max_words=200):
    """Split text into multiple samples of 100-200 words"""
    words = text.split()
    samples = []

    while len(words) >= min_words:
        sample_length = random.randint(min_words, min(max_words, len(words)))
        sample = " ".join(words[:sample_length])
        samples.append(sample)
        words = words[sample_length:]  # Remove used words

    return samples

# Collect 3,000 samples
samples = []
for book_id in book_ids:
    print(f"📖 Extracting book ID: {book_id}")
    text = extract_gutenberg_text(book_id)

    if text:
        book_samples = split_into_samples(text, min_words=100, max_words=200)
        samples.extend(book_samples)

    if len(samples) >= 3000:
        break  # Stop when we reach 3,000 samples

    time.sleep(1)  # Respectful delay

# Limit to exactly 3,000 samples
samples = samples[:3000]

# Save to CSV
filename = "gutenberg_samples.csv"
with open(filename, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["text", "label"])
    writer.writerows([[s, 0] for s in samples])  # Label 0 for human-written text

# Completion message
print(f"\n✅ Successfully saved 3,000 samples to {filename}!")


📖 Extracting book ID: 1342
📖 Extracting book ID: 84
📖 Extracting book ID: 1080
📖 Extracting book ID: 2600

✅ Successfully saved 3,000 samples to gutenberg_samples.csv!


**MACHINE GENERATED TEXT**

In [68]:
from transformers import pipeline
import csv
import random
import time

In [69]:
# Load Hugging Face GPT model
generator = pipeline("text-generation", model="EleutherAI/gpt-neo-2.7B")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Device set to use cpu


In [70]:
# Define categories for automated prompt generation
categories = {
    "news": [
        "Write a news article about {}.",
        "Report on the latest developments in {}.",
        "Summarize the key events in {} today."
    ],
    "opinion": [
        "Write an opinion piece on {}.",
        "Analyze the pros and cons of {}.",
        "Share a perspective on {}."
    ],
    "technology": [
        "Explain the impact of {} on modern life.",
        "Discuss the future of {} in 10 years.",
        "What are the ethical concerns surrounding {}?"
    ],
    "history": [
        "Summarize the historical significance of {}.",
        "Explain how {} changed the world.",
        "Analyze the impact of {} in modern society."
    ],
    "fiction": [
        "Write a short fictional story about {}.",
        "Imagine a world where {} is reality.",
        "Create a sci-fi narrative involving {}."
    ]
}

# Topics for dynamic prompt generation
topics = [
    "climate change", "AI technology", "space exploration", "global politics",
    "mental health awareness", "electric vehicles", "data privacy",
    "cybersecurity", "human rights", "renewable energy", "economic crisis",
    "future of work", "pandemics", "blockchain technology", "social media influence"
]

In [71]:
# Function to generate prompts dynamically
def generate_prompt():
    category = random.choice(list(categories.keys()))
    template = random.choice(categories[category])
    topic = random.choice(topics)
    return template.format(topic)

In [72]:
# Function to generate machine-written text
def generate_text(prompt):
    try:
        response = generator(prompt, max_length=200, num_return_sequences=1)[0]["generated_text"]
        return response.strip()
    except Exception as e:
        print(f"❌ Error generating text: {e}")
        return None

In [73]:
# Generate 6,000 samples
samples = []
for _ in range(6000):
    prompt = generate_prompt()
    text = generate_text(prompt)

    if text:
        samples.append([text, 1])  # Label 1 for machine-generated

    time.sleep(1)  # Avoid overloading API

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

In [3]:
from transformers import pipeline
import csv
import random
import time

# Load GPT model
generator = pipeline("text-generation", model="EleutherAI/gpt-neo-2.7B")

# Categories & Topics for Dynamic Prompts
categories = {
    "news": ["Write a news article about {}.", "Summarize today's events in {}."],
    "opinion": ["Write an opinion piece on {}.", "Discuss the ethical concerns of {}."],
    "technology": ["Explain the impact of {} on society.", "Predict the future of {}."],
    "history": ["Summarize the significance of {}.", "Explain how {} changed the world."],
    "fiction": ["Write a short story about {}.", "Imagine a world where {} is real."]
}

topics = ["climate change", "AI", "space exploration", "global politics", "mental health",
          "electric vehicles", "cybersecurity", "human rights", "renewable energy"]

# Function to generate prompts dynamically
def generate_prompt():
    category = random.choice(list(categories.keys()))
    template = random.choice(categories[category])
    topic = random.choice(topics)
    return template.format(topic)

# Generate 100 samples for testing
samples = []
batch_size = 5  # Generate 5 samples per request
total_samples = 100
requests_needed = total_samples // batch_size

for _ in range(requests_needed):
    prompts = [generate_prompt() for _ in range(batch_size)]

    try:
        responses = generator(prompts, max_length=200, truncation=True, pad_token_id=50256)
        for r in responses:
            text = r[0]["generated_text"].strip()
            if text:
                samples.append([text, 1])  # Label 1 for machine-generated

    except Exception as e:
        print(f"❌ Error generating text: {e}")

    time.sleep(1)  # Avoid overloading

# Save to CSV
filename = "machine_generated_test.csv"
with open(filename, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["text", "label"])
    writer.writerows(samples)

print(f"\n✅ Successfully saved {len(samples)} machine-generated samples to {filename}!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Device set to use cuda:0


KeyboardInterrupt: 

In [5]:
from transformers import pipeline
import csv
import random
import time

# Load Hugging Face's Mistral-7B (Much Faster)
generator = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1")

# Automated Prompt Categories
categories = {
    "news": ["Write a breaking news article about {}.", "Report the latest on {}."],
    "opinion": ["Analyze the effects of {}.", "Write an opinion piece on {}."],
    "science": ["Explain the impact of {} in modern life.", "Summarize recent discoveries in {}."],
    "fiction": ["Write a futuristic sci-fi story involving {}.", "Imagine a world where {} is real."],
}

topics = ["AI ethics", "climate change", "space colonization", "economic collapse",
          "cybersecurity threats", "political instability", "renewable energy", "nuclear fusion",
          "extraterrestrial life", "genetic engineering"]

# Function to generate prompts dynamically
def generate_prompt():
    category = random.choice(list(categories.keys()))
    template = random.choice(categories[category])
    topic = random.choice(topics)
    return template.format(topic)

# Generate 6,000 samples (Fast Mode)
samples = []
batch_size = 10  # Generate 10 samples per request
total_samples = 60
requests_needed = total_samples // batch_size

for _ in range(requests_needed):
    prompts = [generate_prompt() for _ in range(batch_size)]

    try:
        responses = generator(prompts, max_length=200, truncation=True, pad_token_id=50256)
        for r in responses:
            text = r[0]["generated_text"].strip()
            if text:
                samples.append([text, 1])  # Label 1 for machine-generated

    except Exception as e:
        print(f"❌ Error generating text: {e}")

    time.sleep(1)  # Avoid overloading

# Save to CSV
filename = "fast_machine_generated_text.csv"
with open(filename, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["text", "label"])
    writer.writerows(samples)

print(f"\n✅ Successfully saved {len(samples)} fast-generated machine-written samples to {filename}!")

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-v0.1.
401 Client Error. (Request ID: Root=1-67d6a0de-09e6f1ff56a34eb6161814d8;ceb0a501-f12f-479d-bff8-04598a7edab1)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-v0.1 is restricted. You must have access to it and be authenticated to access it. Please log in.

In [2]:
from huggingface_hub import login
login()

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import csv
import random
import time

# Load Mistral-7B Model (Needs login)
model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# Use pipeline for generation
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Define topics & categories for diverse prompts
categories = {
    "news": ["Write a news report on {}.", "Summarize the latest developments about {}."],
    "opinion": ["Analyze the consequences of {}.", "Give your thoughts on {}."],
    "science": ["Explain the significance of {} in modern life.", "Discuss the latest research in {}."],
    "fiction": ["Write a short futuristic story about {}.", "Imagine a world where {} happened."],
}

topics = ["AI ethics", "quantum computing", "climate change", "deep space exploration", "economic crisis",
          "political polarization", "biotechnology", "human evolution", "cybersecurity threats", "virtual reality"]

# Generate prompts
def generate_prompt():
    category = random.choice(list(categories.keys()))
    template = random.choice(categories[category])
    topic = random.choice(topics)
    return template.format(topic)

# Generate 100 samples first
samples = []
batch_size = 10
total_samples = 100
requests_needed = total_samples // batch_size

for _ in range(requests_needed):
    prompts = [generate_prompt() for _ in range(batch_size)]

    try:
        responses = generator(prompts, max_length=200, do_sample=True, temperature=0.8)
        for r in responses:
            text = r[0]["generated_text"].strip()
            if text:
                samples.append([text, 1])  # Label 1 for machine-generated
    except Exception as e:
        print(f"❌ Error: {e}")

    time.sleep(1)

# Save to CSV
filename = "machine_generated_mistral7B.csv"
with open(filename, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["text", "label"])
    writer.writerows(samples)

print(f"\n✅ Successfully saved {len(samples)} machine-generated samples using Mistral-7B!")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [4]:
!pip install transformers torch accelerate sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB ? eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 855.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB ? eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB ? eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 804.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 302.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB ? eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 1.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.5.82
  Attemptin

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load LLaMA 2 model & tokenizer
model_name = "meta-llama/Llama-2-7b-chat-hf"  # You can also try "meta-llama/Llama-2-13b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# Define one test prompt
prompt = "Write a short news article about the latest advancements in AI."

# Encode input
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate output
output = model.generate(**inputs, max_length=200)

# Decode & print result
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("\n📝 Generated Text:\n", generated_text)


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
403 Client Error. (Request ID: Root=1-67d6b78c-591cd48239a626100cb1e7da;dd62c96e-b3de-4cd3-91aa-792b8f9b69f8)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Your request to access model meta-llama/Llama-2-7b-chat-hf is awaiting a review from the repo authors.

In [4]:
from transformers import pipeline

# Load GPT-2 text generation pipeline
generator = pipeline("text-generation", model="gpt2")

# Define a simple prompt
prompt = "The future of artificial intelligence is"

# Generate text
output = generator(prompt, max_length=150, num_return_sequences=1)

# Print the result
print(output[0]["generated_text"])


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The future of artificial intelligence is inextricably bound up with its current status as one of the most important aspects of intelligence.

"We are very close, to the point where we may even be able to run our own software and services without any additional cost," Mr. Kudler said. "Until that software does the job, you have to spend hundreds of millions of dollars to make all the money."

The problem, he said, is that the money goes to a tiny handful of firms, which pay low wages to have their technologies developed. The software can be very expensive, and is often used by bad actors or for malicious purposes.

Even if Google doesn't pay, Mr. Kudler said,


In [7]:
import random
import pandas as pd
from transformers import pipeline

# Load GPT-2 model
generator = pipeline("text-generation", model="gpt2")

# Define categories and templates for prompts
categories = [
    "news",
    "story",
    "essay",
    "science",
    "history",
    "opinion",
    "fantasy",
    "business",
    "technology",
    "politics",
    "environment",
    "health",
    "sports",
    "culture",
    "travel",
    "food",
    "art",
    "literature",
    "economics",
    "education",
    "philosophy",
    "psychology",
    "sociology",
    "design",
    "film",
    "music",
    "gaming",
    "fashion",
    "law",
    "religion",
    "human rights",
    "investigative",
    "documentary",
    "analysis",
    "review",
    "interview",
    "profiles",
    "editorial",
    "commentary",
    "data journalism",
    "multimedia",
    "longform",
    "short story",
    "poetry",
    "satire",
    "feature",
    "development",
    "innovation",
    "geopolitics",
    "climate",
    "sustainability",
    "space",
    "archeology",
    "anthropology"
]

templates = [
    "Write a {category} article about {topic}.",
    "Explain the significance of {topic} in {category}.",
    "Tell a short {category} story about {topic}.",
    "What are the future implications of {topic} in {category}?",
    "Describe a day in the life of someone in {category} focusing on {topic}."
]


# Common topics for randomness
topics = [
    "climate change", "artificial intelligence", "space exploration", "economic crisis",
    "World War II", "ancient civilizations", "robotics in healthcare", "future of education",
    "quantum computing", "mysteries of the deep ocean", "evolution of the internet", "renewable energy",
    "genetic engineering",
    "cybersecurity threats",
    "global pandemics",
    "the rise of cryptocurrency",
    "sustainable agriculture",
    "neuroscience and consciousness",
    "the ethics of AI",
    "colonization of Mars",
    "the impact of social media",
    "ocean acidification",
    "the future of work",
    "virtual reality and augmented reality",
    "the history of aviation",
    "urbanization and smart cities",
    "the refugee crisis",
    "the role of international law",
    "the psychology of decision making",
    "the search for extraterrestrial life",
    "the evolution of music",
    "the effects of deforestation",
    "the future of transportation",
    "data privacy and surveillance",
    "the history of medicine",
    "the art of filmmaking",
    "the impact of automation",
    "the study of black holes",
    "the evolution of language",
    "the challenges of aging populations",
    "the science of climate modeling",
    "the impact of cultural globalization",
    "the history of political revolutions",
    "the physics of quantum entanglement",
    "the role of citizen journalism",
    "the future of food production",
    "the study of human migration",
    "the ethics of animal testing",
    "the impact of wildfires",
    "the development of autonomous vehicles",
    "the study of ancient languages",
    "the future of space tourism",
    "the psychology of addiction",
    "the impact of rising sea levels",
    "the history of photography",
    "the science of earthquakes",
    "the role of artificial organs",
    "the impact of microplastics",
    "the study of exoplanets",
    "the evolution of fashion",
    "the challenges of water scarcity",
    "the history of cryptography",
    "the ethics of gene editing",
    "the impact of space debris",
    "the study of dark matter",
    "the future of personalized medicine"
]

# Function to generate a batch of prompts
def generate_prompts(num_prompts=1000):
    prompts = []
    for _ in range(num_prompts):
        category = random.choice(categories)
        topic = random.choice(topics)
        prompt_template = random.choice(templates)
        prompt = prompt_template.format(category=category, topic=topic)
        prompts.append(prompt)
    return prompts

# Generate 6000 samples
samples = []
prompts = generate_prompts(10)

# Generate text responses
for idx, prompt in enumerate(prompts):
    generated_text = generator(prompt, max_length=150, num_return_sequences=1)[0]["generated_text"]
    samples.append({"text": generated_text, "label": 1})  # Label 1 for machine-generated text

    # Print progress
    if idx % 500 == 0:
        print(f"Generated {idx} samples...")

# Save to CSV
df = pd.DataFrame(samples)
df.to_csv("machine_generated_text.csv", index=False)

print("✅ Successfully generated 6000 machine-generated text samples!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated 0 samples...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


✅ Successfully generated 6000 machine-generated text samples!


In [ ]:
import pandas as pd
import random
from transformers import pipeline

# Load GPT-2 model
generator = pipeline("text-generation", model="gpt2")

# Define categories and templates for prompts
categories = ["news", "story", "essay", "science", "history", "opinion", "fantasy", "business", "technology", "politics", "environment", "health", "sports", "culture", "travel", "food", "art", "literature", "economics", "education", "philosophy", "psychology", "sociology", "design", "film", "music", "gaming", "fashion", "law", "religion", "human rights", "investigative", "documentary", "analysis", "review", "interview", "profiles", "editorial", "commentary", "data journalism", "multimedia", "longform", "short story", "poetry", "satire", "feature", "development", "innovation", "geopolitics", "climate", "sustainability", "space", "archeology", "anthropology"]

templates = [
    "Write a {category} article about {topic}.",
    "Explain the significance of {topic} in {category}.",
    "Tell a short {category} story about {topic}.",
    "What are the future implications of {topic} in {category}?",
    "Describe a day in the life of someone in {category} focusing on {topic}.",
    "Give an in-depth analysis of {topic} within the field of {category}.",
    "Summarize the history and evolution of {topic} in {category}.",
    "What are the ethical concerns related to {topic} in {category}?",
    "How is {topic} expected to change {category} in the next decade?",
    "Compare and contrast different viewpoints on {topic} in {category}."
]

topics = [
    # Science & Technology
    "quantum computing", "dark matter", "human cloning", "AI ethics",
    "nanotechnology", "renewable energy", "5G networks", "data privacy",
    "gene editing", "future of robotics", "electric vehicles", "space tourism",

    # History & Politics
    "World War I", "Cold War", "rise and fall of empires", "industrial revolution",
    "ancient Egyptian civilization", "civil rights movements", "history of democracy",
    "famous historical betrayals", "nuclear arms race", "colonialism and its effects",

    # Economy & Business
    "cryptocurrency regulation", "stock market crashes", "startup culture",
    "future of freelancing", "automation in the workplace", "universal basic income",
    "the future of e-commerce", "gig economy", "the impact of inflation",

    # Health & Society
    "mental health awareness", "work-life balance", "veganism trends",
    "remote work culture", "rise of minimalist lifestyle", "effects of social media",
    "gender equality", "challenges of modern parenting", "impact of aging populations",

    # Environment & Nature
    "climate change policies", "ocean pollution", "wildlife conservation",
    "deforestation crisis", "future of agriculture", "urban sustainability",
    "eco-friendly architecture", "impact of wildfires",

    # Space & Exploration
    "colonizing Mars", "life beyond Earth", "black holes",
    "exoplanets", "the Fermi paradox", "space mining",

    # Random Topics
    "philosophy of free will", "future of education", "importance of reading books",
    "mysteries of the deep sea", "role of luck in success", "sports psychology",
    "music and brain function", "fashion trends over time", "history of magic"
]


def generate_prompts(num_prompts=1000):
    prompts = []
    for _ in range(num_prompts):
        category = random.choice(categories)
        topic = random.choice(topics)
        prompt_template = random.choice(templates)
        prompt = prompt_template.format(category=category, topic=topic)
        prompts.append(prompt)
    return prompts

# Generate prompts
num_samples = 6000 ###CHANGE
batch_size = 20  # Save every 20 samples ###CHANGE
samples = []
prompts = generate_prompts(num_samples)

# Generate and save text
for idx, prompt in enumerate(prompts):
    generated_text = generator(
    prompt,
    max_length=150,
    num_return_sequences=1,
    truncation=True,
    pad_token_id=50256  # EOS token to remove the warning
)[0]["generated_text"]

    samples.append({"text": generated_text, "label": 1})  # Label 1 for machine-generated text

    # Save every 20th samples
    if (idx + 1) % batch_size == 0:
        df = pd.DataFrame(samples)
        df.to_csv("machine_generated_text_final.csv", index=False, mode="a", header=(idx == 0)) ####CHANGE
        print(f"✅ Saved {idx + 1} samples...")

print("✅ Successfully generated and saved 6000 machine-generated text samples!")


Device set to use cpu


✅ Saved 20 samples...
✅ Saved 40 samples...
✅ Saved 60 samples...
✅ Saved 80 samples...
✅ Saved 100 samples...
✅ Saved 120 samples...
✅ Saved 140 samples...
✅ Saved 160 samples...
✅ Saved 180 samples...
✅ Saved 200 samples...
✅ Saved 220 samples...
✅ Saved 240 samples...
✅ Saved 260 samples...
✅ Saved 280 samples...
✅ Saved 300 samples...
✅ Saved 320 samples...
✅ Saved 340 samples...
✅ Saved 360 samples...
✅ Saved 380 samples...
✅ Saved 400 samples...
✅ Saved 420 samples...
✅ Saved 440 samples...
✅ Saved 460 samples...
✅ Saved 480 samples...
✅ Saved 500 samples...
✅ Saved 520 samples...
✅ Saved 540 samples...
✅ Saved 560 samples...
✅ Saved 580 samples...
✅ Saved 600 samples...
✅ Saved 620 samples...
✅ Saved 640 samples...
✅ Saved 660 samples...
✅ Saved 680 samples...
✅ Saved 700 samples...
✅ Saved 720 samples...
✅ Saved 740 samples...
✅ Saved 760 samples...
✅ Saved 780 samples...
✅ Saved 800 samples...
✅ Saved 820 samples...
✅ Saved 840 samples...
✅ Saved 860 samples...
✅ Saved 880 sam

In [38]:
import pandas as pd
import re
from bs4 import BeautifulSoup

# Load the dataset
df = pd.read_csv("machine_generated_text_final.csv", header=None, names=["text", "label"])

In [39]:
# Ensure "text" is a string before processing
df["text"] = df["text"].astype(str)

In [40]:
len(df)

781200

In [41]:
# Remove extra spaces & line breaks
df["text"] = df["text"].apply(lambda x: " ".join(str(x).split()))

In [42]:
len(df)

781200

In [43]:
# Remove non-alphanumeric characters (except punctuation)
import re
df["text"] = df["text"].apply(lambda x: re.sub(r"[^a-zA-Z0-9.,!? ]+", "", x))

In [44]:
len(df)

781200

In [45]:
# Remove exact duplicates
df = df.drop_duplicates()
print("After removing duplicates:", len(df))

After removing duplicates: 5580


In [46]:
print(df.head())

                                                text  label
0  How is space tourism expected to change educat...      1
1  Write a sustainability article about gig econo...      1
2  What are the ethical concerns related to myste...      1
3  Explain the significance of space mining in re...      1
4  Tell a short space story about 5G networks. We...      1


In [72]:
df = df[df["text"].apply(lambda x: len(x.split()) >= 26)]

In [73]:
len(df)

5092

In [74]:
# Save cleaned file
df.to_csv("machine_generated_text_cleaned.csv", index=False)

print("Cleaning complete. Final records:", len(df))

Cleaning complete. Final records: 5092


Cleaning for Human text

In [75]:
import pandas as pd
import re

# Load the CSV files
wikipedia_df = pd.read_csv("wikipedia_texts.csv")
aljazeera_df = pd.read_csv("aljazeera_news_articles.csv")
gutenberg_df = pd.read_csv("gutenberg_samples.csv")

In [78]:
print(wikipedia_df.head())
print(aljazeera_df.head())
print(gutenberg_df.head())

                                                text  label
0  Artificial intelligence  refers to the capabil...      0
1  However, many AI applications are not perceive...      0
2  AI also draws upon psychology, linguistics, ph...      0
3  The emergence of advanced generative AI in the...      0
4  Reasoning and problem-solving Early researcher...      0
                                                text  label
0  Police have arrested several people as part of...      0
1  added. Prosecutors said they could not disclos...      0
2  “To the People of Gaza: A beautiful Future awa...      0
3  To the grieving mothers who have buried their ...      0
4  who claims to stand for peace. This is not jus...      0
                                                text  label
0  *** START OF THE PROJECT GUTENBERG EBOOK 1342 ...      0
1  others who are loved “by allowance” by convent...      0
2  faint in tone, and not enthralling in interest...      0
3  of its composition seen rather more o

In [79]:
# Combine all data
human_df = pd.concat([wikipedia_df, aljazeera_df, gutenberg_df], ignore_index=True)

In [80]:
print(human_df.head())

                                                text  label
0  Artificial intelligence  refers to the capabil...      0
1  However, many AI applications are not perceive...      0
2  AI also draws upon psychology, linguistics, ph...      0
3  The emergence of advanced generative AI in the...      0
4  Reasoning and problem-solving Early researcher...      0


In [81]:
len(human_df)

5152

In [82]:
# Shuffle the data after merging
human_df = human_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [83]:
print(human_df.head())

                                                text  label
0  Political science The application of game theo...      0
1  The arms are dusty regions of star formation w...      0
2  so; for, besides the immediate embarrassment, ...      0
3  However, there remain difficult questions abou...      0
4  charity in a house where such things could be ...      0


In [84]:
# Remove extra spaces and line breaks
human_df["text"] = human_df["text"].apply(lambda x: " ".join(str(x).split()))

In [85]:
# Remove non-alphanumeric characters (except common punctuation)
human_df["text"] = human_df["text"].apply(lambda x: re.sub(r"[^a-zA-Z0-9.,!? ]+", "", x))

In [86]:
# Check the minimum word length in the text column
min_word_count = human_df["text"].apply(lambda x: len(str(x).split())).min()
print(f"Minimum word count in the text column: {min_word_count}")

Minimum word count in the text column: 74


In [88]:
# Check the minimum word length in the text column
min_word_count = human_df["text"].apply(lambda x: len(str(x).split())).max()
print(f"Max word count in the text column: {min_word_count}")

Max word count in the text column: 200


In [89]:
# Save to a new file
human_df.to_csv("human_written_texts_cleaned.csv", index=False)

COMBINING THE TWO DATASETS (MACHINE AND HUMAN)

In [90]:
# Load the cleaned datasets
human_df = pd.read_csv("human_written_texts_cleaned.csv")
machine_df = pd.read_csv("machine_generated_text_cleaned.csv")

In [91]:
print("Human Data Labels - Min:", human_df["label"].min(), "Max:", human_df["label"].max())
print("Machine Data Labels - Min:", machine_df["label"].min(), "Max:", machine_df["label"].max())

Human Data Labels - Min: 0 Max: 0
Machine Data Labels - Min: 1 Max: 1


Human data with Label : 0 and Machine data with Label: 1

In [92]:
# Combine both datasets
combined_df = pd.concat([human_df, machine_df], ignore_index=True)

In [93]:
# Shuffle the dataset to mix human and machine-generated text
combined_df = combined_df.sample(frac=1, random_state=72).reset_index(drop=True)

In [94]:
print(combined_df.head())

                                                text  label
0  In the Vigenre cipher, a polyalphabetic cipher...      0
1  the carriage came to the door, my uncle was ca...      0
2  some partiality, especially on Lydias side, bu...      0
3  her there, Natsha ran to the nursery, but Snya...      0
4  What are the ethical concerns related to unive...      1


In [95]:
len(combined_df)

10244

In [96]:
# Save the final dataset
combined_df.to_csv("final_combined_dataset.csv", index=False)